In [1]:
import numpy as np
import pandas as pd
import os,sys
from sklearn import linear_model
from scipy import stats as stats

In [2]:
# load lb, test and CV CIDs

# load LB CIDs
with open('/media/gabor/H/python_from_C/final_ofaction_for_paper_2/data/CID_leaderboard.txt') as f: 
    content = f.readlines()
lb_CIDs = list(content)  
lb_CIDs = [int(x) for x in lb_CIDs]

# load test CIDs
with open("/media/gabor/H/python_from_C/final_ofaction_for_paper_2/data/CID_testset.txt") as f: 
    content = f.readlines()
test_CIDs = list(content)  
test_CIDs = [int(x) for x in test_CIDs]



In [4]:
# load morgan matrix to use them as weights in training
morgan = pd.read_csv('/media/gabor/H/python_from_C/final_ofaction_for_paper_2/data/morgan_sim.csv', index_col=0)
weights = morgan[morgan.index.astype(str)]
weights = pd.concat((weights,weights)).sort_index()
print weights.shape
weights.head()

(952, 476)


,126,176,177,180,196,239,240,241,243,244,...,5366244,5367698,5367706,5368076,5371102,6114390,6429333,6999977,10857465,16220109
0,,,,,,,,,,,,,,,,,,,,,
126,1.000000,0.108108,0.171429,0.054054,0.066667,0.090909,0.509091,0.166667,0.315789,0.290909,...,0.033613,0.183673,0.195652,0.051948,0.252632,0.237288,0.307692,0.066667,0.000000,0.050633
126,1.000000,0.108108,0.171429,0.054054,0.066667,0.090909,0.509091,0.166667,0.315789,0.290909,...,0.033613,0.183673,0.195652,0.051948,0.252632,0.237288,0.307692,0.066667,0.000000,0.050633
176,0.108108,1.000000,0.285714,0.625000,0.256410,0.434783,0.058824,0.000000,0.277778,0.058824,...,0.081633,0.103896,0.112676,0.142857,0.135135,0.082474,0.136364,0.256410,0.027397,0.172414
176,0.108108,1.000000,0.285714,0.625000,0.256410,0.434783,0.058824,0.000000,0.277778,0.058824,...,0.081633,0.103896,0.112676,0.142857,0.135135,0.082474,0.136364,0.256410,0.027397,0.172414
177,0.171429,0.285714,1.000000,0.285714,0.054054,0.095238,0.187500,0.000000,0.058824,0.000000,...,0.041667,0.080000,0.086957,0.111111,0.083333,0.084211,0.238095,0.108108,0.028169,0.142857


In [3]:
#load the features
features = pd.read_csv('features_dragon_morgan.csv', index_col=0)
features.head()

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,91541756_2,91552833_2,91563027_2,91595028_2,91614181_2,91617014_2,91617930_2,91618238_2,neglog10d,Intensity
0,126,0.181128,0.270753,0.030587,0.262264,0.219126,0.253846,0.214989,0.216981,0.425532,...,0.014024,0.000296,0.021098,0.000186,0.003159,0.002299,0.000138,0.011080,1,1
1,126,0.181128,0.270753,0.030587,0.262264,0.219126,0.253846,0.214989,0.216981,0.425532,...,0.014024,0.000296,0.021098,0.000186,0.003159,0.002299,0.000138,0.011080,3,0
2,176,0.060311,0.109331,0.025411,0.096943,0.105579,0.090940,0.107335,0.125214,0.659574,...,0.008391,0.000930,0.001442,0.000094,0.000607,0.001362,0.000229,0.004162,5,1
3,176,0.060311,0.109331,0.025411,0.096943,0.105579,0.090940,0.107335,0.125214,0.659574,...,0.008391,0.000930,0.001442,0.000094,0.000607,0.001362,0.000229,0.004162,7,0
4,177,0.020039,0.067721,0.015501,0.075556,0.083688,0.078074,0.089782,0.106346,0.382979,...,0.000961,0.000339,0.000657,0.000008,0.000098,0.000221,0.000037,0.001932,3,1


In [46]:
]

In [7]:
# give a number for each descriptor
descriptor = {}
for idx, desc in enumerate([u'INTENSITY/STRENGTH', u'VALENCE/PLEASANTNESS', u'BAKERY', 
                       u'SWEET', u'FRUIT', u'FISH', u'GARLIC', u'SPICES', u'COLD', u'SOUR', u'BURNT',
                       u'ACID', u'WARM', u'MUSKY', u'SWEATY', u'AMMONIA/URINOUS', u'DECAYED', u'WOOD',
                       u'GRASS', u'FLOWER', u'CHEMICAL']):
    descriptor[idx] = desc

In [8]:
# load the targets
all_targets = pd.read_csv('target.csv', index_col=0)
all_targets.head()

,#oID,individual,INTENSITY/STRENGTH,VALENCE/PLEASANTNESS,BAKERY,SWEET,FRUIT,FISH,GARLIC,SPICES,...,ACID,WARM,MUSKY,SWEATY,AMMONIA/URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,126,25,49.551020,49.465116,0.674419,25.953488,6.581395,0.302326,1.720930,3.906977,...,3.046512,0.790698,8.023256,1.604651,1.209302,5.069767,1.348837,1.441860,9.906977,14.813953
1,126,25,24.653061,49.465116,0.674419,25.953488,6.581395,0.302326,1.720930,3.906977,...,3.046512,0.790698,8.023256,1.604651,1.209302,5.069767,1.348837,1.441860,9.906977,14.813953
2,176,25,11.551020,45.944444,3.666667,8.166667,1.777778,0.000000,10.388889,6.055556,...,4.166667,6.111111,8.666667,2.166667,5.222222,4.388889,2.611111,2.166667,5.944444,4.222222
3,176,25,4.551020,45.944444,3.666667,8.166667,1.777778,0.000000,10.388889,6.055556,...,4.166667,6.111111,8.666667,2.166667,5.222222,4.388889,2.611111,2.166667,5.944444,4.222222
4,177,25,33.265306,45.147059,9.411765,22.441176,1.676471,0.000000,0.705882,2.735294,...,4.970588,4.470588,3.823529,2.176471,4.235294,3.558824,1.147059,4.470588,2.441176,18.794118


In [9]:
scores = pd.read_csv('LB_scores/scores_' + str(0) + '.csv',index_col=0)

In [9]:
#load splits
trainsplits = pd.read_csv('/media/gabor/H/python_from_C/final_ofaction_for_paper_2/data/cv_splits_train_big.csv',header=None)
testsplits = pd.read_csv('/media/gabor/H/python_from_C/final_ofaction_for_paper_2/data/cv_splits_test_big.csv',header=None)

In [20]:
# predict LB with different number of features
for k in range(10):
    print k
    # set a cv split as holout data
    lb_CIDs = testsplits.ix[k,:].values
    #for feature_number in [5,10,20,50,100,200,300,500,1000,1500,2000,3000,4000]:
    #for feature_number in [1,2,3,5,10,33,100,333,1000,3333,10000]:
    for feature_number in [1,2,3,4,5,10,33,100,333,1000,3333,10000]: #new run, with 1,2,3 features
        print(feature_number)
        sys.stdout.flush()

        targets = all_targets[~all_targets['#oID'].isin(test_CIDs)]# remove test data 
        features = features[~features.CID.isin(test_CIDs)] # remove test data 

        train_targets = targets[~targets['#oID'].isin(lb_CIDs)]  # exclude lb targets from training
        train_features = features[~features.CID.isin(lb_CIDs)] # exclude lb features from training
        test_features = features[features.CID.isin(lb_CIDs)] 

        # set the regressor
        regr = linear_model.Ridge(alpha=1, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='auto')


        result = []
        for idx in range(21):

            #print(descriptor[idx])

            # load the scores for the descriptor
            scores = pd.read_csv('LB_scores_morgan' + str(k) + '/scores_' + str(idx) + '.csv',index_col=0)

            #exclude the data leak 
            scores = scores.loc[[x for x in scores.index if x not in['Intensity','neglog10d']] ].sort_values(by='0', ascending=0)

            X_all = train_features[scores.sort_values(by='0',ascending=0)[:feature_number].index] # set X values with the best features
            X_all['CID'] = train_features.CID # add the CIDs as a column
            

            for CID in lb_CIDs:

                Y_train = train_targets[['#oID',descriptor[idx]]]

                Y_train = Y_train[~Y_train[descriptor[idx]].isnull()]
                X = X_all[X_all.CID.isin(Y_train['#oID'])]
                weight = weights[weights.index.isin(Y_train['#oID'])][str(CID)]

                if idx == 0: # if predicting intensity, use 1/1000 dilutions (neglog10 of 1/1000 is 3)
                    test_data = test_features[test_features.neglog10d == 3]
                    test_data = test_data[test_data.CID == CID]
                    test_data = test_data[scores.sort_values(by='0',ascending=0)[:feature_number].index]

                else: # otherwise use high dilution data (not that they differ in this target matrix from the low ones)
                    test_data = test_features[test_features.Intensity == 1]
                    test_data = test_data[test_data.CID == CID]
                    test_data = test_data[scores.sort_values(by='0',ascending=0)[:feature_number].index]
                    
                    
                # in case the data frame lenght is zero, dont try to predict

                if len(test_data) == 0:
                    print 'zero data',CID
                else:
                    regr.fit(X.drop('CID',1),Y_train[descriptor[idx]], sample_weight = weight.values)

                    Y_test = regr.predict(test_data)
                    std = -(Y_test**2)/2500.0+Y_test/25.0
                    result.append([CID, descriptor[idx], Y_test,std])

        result = pd.DataFrame(result)
        result.columns = ['#oID', 'descriptor', 'value', 'sigma']

        result.value = result.value.astype(float)
        result.sigma = result.sigma.astype(float)

        # remove negative data and data above 100

        result.value[result.value < 0] = 0 
        result.value[result.value > 100] = 100

        result.sigma[result.sigma < 0] = 0


        #result_mean['sigma'] = -(result_mean.value**2)/2500.0+result_mean.value/25.0
        result.to_csv('results_morgan_noleak/' + str(k) + '/subchallenge2_' +str(feature_number) + '.txt',sep='\t',index =0)


0
4


/home/gabor/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


zero data 6501
zero data 8063
zero data 7991
zero data 556940
zero data 61199
zero data 176
zero data 263
zero data 61130
1
4


/home/gabor/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gabor/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gabor/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


zero data 18635
zero data 1549026
zero data 61945
zero data 23642
zero data 31252
zero data 7341
zero data 6920
zero data 7969
2
4
zero data 9609
zero data 7969
zero data 7894
zero data 61199
zero data 61130
zero data 7916
3
4
zero data 7916
zero data 10430
zero data 9016
zero data 12587
zero data 15717
zero data 18635
zero data 526618
zero data 7894
4
4
zero data 10430
zero data 10448
zero data 7991
zero data 61653
zero data 15380
zero data 101010
zero data 27458
zero data 10285
zero data 62725
zero data 61527
5
4
zero data 7150
zero data 556940
zero data 8118
zero data 6590
zero data 6429333
zero data 61641
zero data 7969
zero data 5362588
zero data 31252
zero data 61130
zero data 18635
zero data 61199
6
4
zero data 10797
zero data 10430
zero data 526618
zero data 7991
zero data 7894
zero data 263
zero data 5362588
zero data 18635
zero data 61130
zero data 9016
zero data 1032
7
4
zero data 61527
zero data 1549026
zero data 7144
zero data 263
zero data 10430
zero data 6501
zero data 5

In [29]:
test_features[test_features.CID == CID]

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,Hypertens-80_2,Hypertens-50_2,Hypnotic-80_2,Hypnotic-50_2,Neoplastic-80_2,Neoplastic-50_2,Infective-80_2,Infective-50_2,neglog10d,Intensity
308,7916,0.183463,0.213539,0.018611,0.218204,0.217213,0.218838,0.221028,0.132933,0.375887,...,0,0,0,0,0,0,0,0,5,1
309,7916,0.183463,0.213539,0.018611,0.218204,0.217213,0.218838,0.221028,0.132933,0.375887,...,0,0,0,0,0,0,0,0,7,0


In [22]:
test_data = test_features[test_features.neglog10d == 3]
#test_data = test_data[test_data.CID == CID]
#test_data = test_data[scores.sort_values(by='0',ascending=0)[:feature_number].index]
test_data

,CID,complexity from pubmed,MW,AMW,Sv,Se,Sp,Si,Mv,Me,...,Hypertens-80_2,Hypertens-50_2,Hypnotic-80_2,Hypnotic-50_2,Neoplastic-80_2,Neoplastic-50_2,Infective-80_2,Infective-50_2,neglog10d,Intensity
11,239,0.102724,0.184880,0.020081,0.173157,0.187816,0.169183,0.193197,0.108919,0.503546,...,0,0,0,0,0,0,0,0,3,0
14,241,0.030156,0.156299,0.017265,0.189579,0.158847,0.199864,0.165365,0.186964,0.078014,...,0,0,0,0,0,0,0,0,3,1
33,325,0.196498,0.343857,0.013183,0.393635,0.371028,0.411944,0.381254,0.138937,0.148936,...,0,0,0,0,0,0,1,0,3,0
38,379,0.173735,0.328253,0.009412,0.370962,0.390989,0.389775,0.404832,0.095197,0.205674,...,0,0,0,0,0,0,0,0,3,1
68,1001,0.126459,0.268335,0.013591,0.311468,0.288660,0.327932,0.300793,0.144940,0.113475,...,0,0,0,0,0,0,0,0,3,1
92,2346,0.256809,0.341231,0.035778,0.315476,0.244267,0.331467,0.244026,0.239280,0.212766,...,0,0,0,0,0,0,1,0,3,1
113,5541,0.445525,0.520675,0.025550,0.470627,0.464004,0.447941,0.454528,0.146655,0.567376,...,0,0,0,0,1,0,1,0,3,0
130,6050,0.591440,0.739597,0.016644,0.744646,0.749431,0.746776,0.752025,0.121784,0.354610,...,1,1,0,0,1,1,1,0,3,1
134,6057,0.342412,0.424399,0.025754,0.405540,0.370847,0.394752,0.367998,0.174099,0.432624,...,0,0,1,0,1,0,1,0,3,1
145,6213,0.165953,0.197987,0.033990,0.148211,0.154410,0.159060,0.155754,0.127787,0.531915,...,0,0,0,0,0,0,0,0,3,0


In [16]:
CID

6501